# Notes

for the leoc96, I have to provide the left most values as the lon cordinates, instead of the center point, because if not it causes a crash 
for the gfdl (2pm) data we have to check this

Do I have to do this for all model data? 
If so, I might have to hard fix the crash that occurs when plotting using basemap. 

In [1]:
import numpy as np 
import xarray as xr 
import matplotlib.pyplot as plt 
import cartopy
import os
import pandas as pd
import cftime
import datetime as dt
import tqdm
import netCDF4 as nc

import os 
import glob

In [2]:
# reading in LEOC96 data
in_folder = '/mnt/drive3/gfdl/6HRLY/SURF/'

var = 'wap'
var_map = {'wap': 'wap'}
var_long_name = {'wap': 'Vertical Velocity'}
var_sn = {'wap': 'lagrangian_tendency_of_air_pressure_500'}
var_units = {'wap': 'Pa/s'}
out_var_map = {'wap': 'wap'}

var_scale = {'wap': 1.} 

in_file = f'/mnt/drive3/gfdl/6HRLY/KLEV/atmos.2008010100-2012123123.{var_map[var]}.nc'
print(os.path.exists(in_file))

True


In [3]:
ds = xr.open_dataset(in_file)
lat_dim = ds.lat.size
lon_dim = ds.lon.size
dims = ds[var_map[var]].shape[1:]
print(dims)
ds.close()

(32, 180, 288)


In [4]:
print(ds.variables[var_map[var]])
print(ds.phalf)

<xarray.Variable (time: 7308, pfull: 32, lat: 180, lon: 288)>
[12123095040 values with dtype=float32]
Attributes:
    long_name:      omega
    units:          Pa/s
    cell_methods:   time: point
    interp_method:  conserve_order2
<xarray.DataArray 'phalf' (phalf: 33)>
array([   1.      ,    4.      ,    8.186021,   13.788865,   20.917952,
         29.836408,   41.217896,   55.792215,   74.201906,   97.047864,
        124.966648,  158.549553,  198.396959,  245.027221,  298.888576,
        360.040179,  427.458025,  498.243573,  568.220535,  633.836047,
        693.266329,  745.991986,  792.097373,  831.921945,  865.977814,
        894.872525,  919.22792 ,  939.635932,  956.672132,  970.827661,
        982.570665,  992.23    , 1000.      ])
Coordinates:
  * phalf    (phalf) float64 1.0 4.0 8.186 13.79 ... 970.8 982.6 992.2 1e+03
Attributes:
    long_name:       ref half pressure level
    units:           mb
    cartesian_axis:  Z
    positive:        down


In [5]:
in_lat = ds['lat'].values
in_lon = ds['lon'].values
in_var = ds[var_map[var]]

In [14]:
# year_range = [2008, 2012]
year_range = [2008, 2009]

time_start_year = 1975

# must include the end year, so have to add one 
num_years = year_range[1] - year_range[0] + 1

# creating the output arrays
out_time_bnds = np.zeros((1460*num_years, 2))

# time variable for all the years
in_time = np.arange(0, num_years*365, .25) + (year_range[0] - time_start_year)*365 
print(len(in_time))

2920


In [21]:
out_data = in_var.isel(time=range(0,len(in_time)))
print(out_data.shape)

(2920, 32, 180, 288)


In [ ]:
out_var = np.array(out_data, dtype=np.float32)
out_time = in_time

In [ ]:
print(out_var.shape)

In [25]:
lon = np.copy(in_lon)
dlon = lon[1] - lon[0]
out_lon = np.round(lon - dlon/2., decimals=2)

lat = np.copy(in_lat)
dlat = lat[1] - lat[0]
out_lat = np.round(lat - dlat/2., decimals=2)

# getting the output time bounds
out_time_bnds[:, 1] = out_time
out_time_bnds[0, 0] = out_time[0]
out_time_bnds[1:, 0] = out_time[:-1]

print(out_time_bnds.shape, out_lat.shape, out_lon.shape, out_var.shape)

(7300, 2) (180,) (288,) (7300, 180, 288)


## Creating the date variable provided in model data

In [26]:
# time_arr = [cftime.DatetimeNoLeap(time_start_year, 1, 1) + dt.timedelta(days=i) for i in tqdm.notebook.tqdm(out_time, total=len(out_time))]
out_date = np.empty(out_time.shape, dtype=int)
for i, i_time in tqdm.tqdm_notebook(enumerate(out_time), total=len(out_time)): 
    tmp = cftime.DatetimeNoLeap(time_start_year, 1, 1) + dt.timedelta(days=i_time)
    out_date[i] = int('%04d%02d%02d'%(tmp.year, tmp.month, tmp.day))

/home/jj/anaconda3/envs/mcms/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
print(out_date)

[20080101 20080101 20080101 ... 20121231 20121231 20121231]


# Creating the model sample data for the SLP values

In [28]:
# write the output as netcdf file 

# out_file = '/localdrive/drive10/jj/mdtf/inputdata/model/QBOi.EXP1.AMIP.001/6hr/QBOi.EXP1.AMIP.001.SLP.6hr.nc'
# out_file = '/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.SLP.6hr.nc'

# out_file = '/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.SLP.6hr.nc'
out_file = f'/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.{out_var_map[var]}.6hr.nc'
print(out_file)

# # Format of the PRECT variable for 3hr is as follows: 
# 1. time - noleap, days since 1975-01-01 00:00:00
# 2. date - current date
# 3. lat - latitude
# 4. lon - longitude
# 5. time_bnds - time interval endpoints
# 6. SLP - mba, long_name, cell_methods: "time:mean"

out_ds = xr.Dataset({
        'time_bnds': (('time', 'nbnd'), out_time_bnds), 
        'date': (('time'), out_date),
        out_var_map[var]: (('time', 'lat', 'lon'), out_var), 
    }, 
    coords={
        'time': out_time, 
        'lat': out_lat, 
        'lon': out_lon,
        'nbnd': [1, 2]
    },
    )

# lon attribs
out_ds.lon.attrs['long_name'] = 'longitude'
out_ds.lon.attrs['units'] = 'degrees_east'

# lat attribs
out_ds.lat.attrs['long_name'] = 'latitude'
out_ds.lat.attrs['units'] = 'degrees_north'

# time attribs
out_ds.time.attrs['long_name'] = 'time'
out_ds.time.attrs['units'] = 'days since 1975-01-01 00:00:00'
out_ds.time.attrs['calendar'] = 'noleap'
out_ds.time.attrs['bounds'] = 'time_bnds'

# time_bnds attribs
out_ds.time_bnds.attrs['long_name'] = 'time interval endpoints'

# date attribs
out_ds.date.attrs['long_name'] = 'current date (YYYYMMDD)'

# slp attribs
out_ds[out_var_map[var]].attrs['standard_name'] = var_sn[var]
out_ds[out_var_map[var]].attrs['long_name'] = var_long_name[var]
out_ds[out_var_map[var]].attrs['units'] = var_units[var]

/localdrive/drive10/jj/mdtf/inputdata/model/GFDL.EXP1.2PM.001/6hr/GFDL.EXP1.2PM.001.wap500.6hr.nc


In [46]:
# actually writing the output file
out_ds.to_netcdf(out_file)